In [19]:
### Build simple Neuron Network to apply mnist recognization 

# 參考 李弘毅老師影片 ML Lecture 8-1: “Hello world” of deep learning
# https://www.youtube.com/watch?v=Lx3l4lOrquw&list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&index=13

In [54]:
!pip list | grep Keras

Keras (2.1.2)


DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.


In [20]:
import keras
from keras.layers import Activation, Dense
import numpy as np
import pandas as pd

In [21]:
from keras.datasets import mnist  # mnist dataset is built-in in Keras module

In [42]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [23]:
x_train.shape

(60000, 28, 28)

In [24]:
y_train.shape

(60000,)

In [25]:
x_test.shape

(10000, 28, 28)

In [26]:
y_test.shape

(10000,)

In [27]:
### model initial
model = keras.Sequential()

In [28]:
# first layer
model.add( Dense( input_dim=28*28, output_dim=500))

C:\Users\edwardchen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, units=500)`
  


In [29]:
model.add( Activation("sigmoid"))

In [30]:
model.add( Dense(output_dim=500))  # 第二層不用給input了
model.add( Activation("sigmoid"))

C:\Users\edwardchen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=500)`
  """Entry point for launching an IPython kernel.


In [31]:
# 設定輸出層
model.add( Dense(output_dim = 10))
model.add( Activation("softmax"))

C:\Users\edwardchen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10)`
  


In [48]:
### Define goodness of the function
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
### training begin
model.fit(x_train, y_train, batch_size=100, nb_epoch=20)

C:\Users\edwardchen\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected dense_1_input to have 2 dimensions, but got array with shape (60000, 28, 28)

In [43]:
### 修改 input 型態 
# x_train (60000, 28, 28) -> (60000, 28*28) 把 grid 攤平
x_train = x_train.reshape((60000, 28*28))
x_test = x_test.reshape((10000, 28*28))
# y_train (60000,) -> (60000, 10) 做 one-hot encode
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [44]:
x_train.shape

(60000, 784)

In [45]:
y_train.shape

(60000, 10)

In [49]:
### restrart our training
model.fit(x_train, y_train, batch_size=100, nb_epoch=20) #

C:\Users\edwardchen\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/20
60000/60000 [==============================] - 9s 156us/step - loss: 0.3909 - acc: 0.8893
Epoch 2/20
60000/60000 [==============================] - 8s 142us/step - loss: 0.2627 - acc: 0.9208
Epoch 3/20
60000/60000 [==============================] - 9s 145us/step - loss: 0.2389 - acc: 0.9269
Epoch 4/20
60000/60000 [==============================] - 8s 141us/step - loss: 0.2317 - acc: 0.9295
Epoch 5/20
60000/60000 [==============================] - 9s 143us/step - loss: 0.2102 - acc: 0.9347
Epoch 6/20
60000/60000 [==============================] - 8s 136us/step - loss: 0.1994 - acc: 0.9392
Epoch 7/20
60000/60000 [==============================] - 8s 133us/step - loss: 0.1821 - acc: 0.9426
Epoch 8/20
60000/60000 [==============================] - 8s 138us/step - loss: 0.1755 - acc: 0.9456
Epoch 9/20
60000/60000 [==============================] - 8s 137us/step - loss: 0.1649 - acc: 0.9477
Epoch 10/20
60000/60000 [==============================] - 8s 134us/step - loss: 0.1625 - a

In [ ]:
# batch_size : training data divide into mini-batch  ex. 60000 pics, batch size 100 -> 600 parts, i.e. update 600 times 
# in one epoch. Keras do this with randomly choose from the training set, it should be.
# nb_epoch : how many times you want to do with the above process. Thus, we have 600 * 20 updates in this case.

# if batch_size = 1 --> stochastic gradient decent(sgd) (60000 * 20 次 updates) 
# 不過要考量到實做時的速度，batch size 1, 10, 100之間花的時間差異並不是線性的。
# ex. 李弘毅老師用GTX980跑 size=1: 166s/epoch; size=10: 17s/epoch; size=100: 1s/epoch  
# size大時會快，是因為GPU的平行運算，不過size太大，應該就沒有那麼多核心作平行運算。 
# ex. 一百個核心處理1個mission跟處理100個mission速度差不多，但是處理101個mission速度應該就要兩倍了

# 166s ~= 17s * 10  在用sgd時，update 60000 *1epoch，跟batch size 10時，update 6000 * 10 的時間一樣，update的次數也一樣。
# 所以會選擇 size = 10 的，因為比較穩定。
# sgd每次update只看一個dim方向，好處是快速update，但現實層面沒有比較快的情況下，會選擇穩定的。

In [51]:
### apply testing set
score = model.evaluate(x_test, y_test)
score    # total loss & Accuracy

10000/10000 [==============================] - 1s 66us/step


[0.13320024986490608, 0.95660000000000001]

In [53]:
### save model
model.save("./mnist_Keras_model")   # about 7000 KB

In [ ]:
### if there's a new data
result = model.predict(x_new)